In [9]:

import json 
import threading
import time

from joblib import load
import pandas as pd
from datetime import datetime

from flask import Flask, render_template
from flask_cors import CORS

from api_scripts.binance import get_binance_prices 
from api_scripts.coin_gecko import get_coingecko_prices 
from api_scripts.coin_layer import get_coinlayer_prices, get_coinlayer_prices_yesterday 

In [10]:

def format_cgd(cgd):
    dataf = {}
    for coin in cgd:
        if(coin["Symbol"]=="btc"):
             dataf["BTC"] = coin["Price"]
        if(coin["Symbol"]=="ltc"):
             dataf["LTC"] = coin["Price"]
        if(coin["Symbol"]=="ada"):
             dataf["ADA"] = coin["Price"]
        if(coin["Symbol"]=="eth"):
             dataf["ETH"] = coin["Price"]
        if(coin["Symbol"]=="usdt"):
             dataf["USDT"] = coin["Price"]
    
    return dataf
    
def format_bd(bd):
        return {"BTC":float(bd["BTCUSD"]),"LTC":float(bd["LTCUSD"]),"ADA":float(bd["ADAUSD"]),"ETH":float(bd["ETHUSD"]),"USDT":float(bd["USDTUSD"]),}

def opportunity(buy,price,sell,cost,coin):
    return{"coin":coin,
    "sellerMarket":buy,
    "buyPrice":price,
    "buyerMarket":sell,
    "sellPrice":cost }
    
def compare(m1,m2,coin):
    m1price = m1["data"][coin]
    m2price = m2["data"][coin]
    if(m1price != m2price): 
        if(m1price< m2price): 
            return opportunity(m1["Name"],m1price,m2["Name"],m2price,coin)
        else:
            return opportunity(m2["Name"],m2price,m1["Name"],m1price,coin)
    else: 
        return False
    
def generate_arbitrage_opportunities(stale):
    if stale == False:
        cgd = get_coingecko_prices()
        bd = get_binance_prices()
        clfd = get_coinlayer_prices()
        # clfd = json.loads(get_coinlayer_prices())["rates"]
    else:
        print("stale")
        with open("jsons/cgarb.json","r") as f:
            cgd = json.load(f)
        with open("jsons/barb.json","r") as f:
            bd = json.load(f)
        with open("jsons/clarb.json","r") as f:
            clfd = json.load(f)["rates"]

    clf = {"Name":"CoinLayer","data":clfd}
    cgf = {"Name":"CoinGecko","data":format_cgd(cgd)}
    bf = {"Name":"Binance","data":format_bd(bd)}

    opportunities = []
    for coin in clf["data"]:
        comps = [ compare(clf,cgf,coin),
             compare(clf,bf,coin),
             compare(cgf,bf,coin)]
            
        for comp in comps:
            if comp != 0:
                opportunities.append(comp)
    return opportunities

def format_yesterdays_data(stale):
    if stale == True:
        with open("jsons/clarbm2.json","r") as cl:
            yesterdata = json.load(cl)
    else:
        yesterdata = get_coinlayer_prices_yesterday()
        
    yd = datetime.strptime(yesterdata["date"], "%Y-%m-%d")
    yestercoins = yesterdata["rates"]
    formatted_data={}
    for coin in yestercoins:
        formatted_data[coin] = {
        "Open":[yestercoins[coin]["rate"]],
        "High":[yestercoins[coin]["high"]],
        "Low":[yestercoins[coin]["low"]],
        "Close":[yestercoins[coin]["rate"]],
        "Adj Close":[yestercoins[coin]["rate"]],
        "Volume":[yestercoins[coin]["vol"]],
        "year":[yd.year],
        "month":[yd.month],
        "day":[yd.day],}
    return formatted_data

def retrieveModels():
    coins = ['BTC','LTC','ETH','ADA','USDT']
    mls = {}
    for coin in coins:
        mls[coin] = load(f"ml/regressor_{coin}.joblib") 
    return mls

def model_preds():
    final_pred = {}
    coin_data = data2["pred_data"]
    for coin in coin_data:
        final_pred[coin] = models[coin].predict(pd.DataFrame(coin_data[coin]))
    return final_pred


In [11]:
app = Flask(__name__)

CORS(app,resources={r"/*": {"origins": ["https://turkishfilms.github.io"]}})

models = retrieveModels()

data2 = {"data":generate_arbitrage_opportunities(True),"predData":format_yesterdays_data(True)}

def get_data_on_delay():
    while True:
        data2["data"] = generate_arbitrage_opportunities(True)
        data2["pred_data"] = format_yesterdays_data(True)
        time.sleep(30) 

@app.route('/')
def home():
   return render_template('index.html',data=data2["data"])

@app.route('/learn')
def a():
    return render_template('learn.html')

@app.route('/predictions')
def b():
    return render_template('predictions.html',data=json.dumps(model_preds()))

if(__name__ == "__main__"):
    t = threading.Thread(target=get_data_on_delay)
    t.start()
    app.run()

stale
stale
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-5 (get_data_on_delay):
Traceback (most recent call last):
  File "d:\programs\python\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "d:\programs\python\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ddram\AppData\Local\Temp\ipykernel_1452\781838273.py", line 11, in get_data_on_delay
  File "C:\Users\ddram\AppData\Local\Temp\ipykernel_1452\1696228769.py", line 59, in generate_arbitrage_opportunities
  File "C:\Users\ddram\AppData\Local\Temp\ipykernel_1452\1696228769.py", line 29, in compare
KeyError: 'success'


stale
